In [1]:
from google.colab import drive
drive.mount("/content/drive")
%cd "/content/drive/MyDrive/Colab Notebooks/Master_homework/Advanced_programming_for_scientific_computing"

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/Master_homework/Advanced_programming_for_scientific_computing


/content/drive/MyDrive/Colab Notebooks/Master_homework/Advanced_programming_for_scientific_computing


In [2]:
import os
import imageio
from glob import glob
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import numpy as np
from tqdm import tqdm

In [ ]:
im_list = glob("Dulieu/*.jpeg")
save_dir = "cropped"
os.makedirs(save_dir, exist_ok = True)
for img_file in tqdm(im_list):
  basename = os.path.basename(img_file).split(".")[0]
  img = imageio.imread(img_file)

  value_channel = cv2.cvtColor(img, cv2.COLOR_RGB2HSV )[:,:, 2]
  harris = cv2.cornerHarris(value_channel,2,3,0.01)  # Applies harris corner detector to gray image

      # Result is dilated for marking the corners, not important
  # harris = cv2.dilate(harris,None)

  # Threshold for an optimal value, it may vary depending on the image.
  mask = harris>0.01*harris.max()

  # Finding Contours
  # Use a copy of the image e.g. edged.copy()
  # since findContours alters the image
  contours, hierarchy = cv2.findContours(mask.astype(np.uint8),
      cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

  points = np.concatenate(contours).squeeze()

  min_x, max_x = min(points[:, 0]), max(points[:, 0])
  min_y, max_y = min(points[:, 1]), max(points[:, 1])
  img_cropped = img[min_y:max_y, min_x:max_x]
  # plt.imshow(img_cropped)
  # plt.title("Harris")
  # plt.xticks([]), plt.yticks([])
  # plt.savefig('Harris_detector', bbox_inches='tight')
  # plt.show()

  uri = os.path.join(f"{save_dir}/{basename}_cropped.jpg")
  imageio.imwrite(uri, img_cropped)

  0%|          | 0/8989 [00:00<?, ?it/s]<ipython-input-68-39b40637e467>:6: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img = imageio.imread(img_file)
100%|██████████| 8989/8989 [03:14<00:00, 46.12it/s]


In [3]:
cropped_list = glob("cropped/*.jpg")
name_set = []
for cropped in cropped_list:
  name = cropped.split("/")[1].split("_")[0]
  name_set.append(name)

In [4]:
name_set = list(set(name_set))
name_set.sort()

In [5]:
# name = name_set[0]
name_and_number = {}
for name in name_set:
  img_list = glob(f"cropped/{name}*_cropped.jpg")
  name_and_number[name] = len(img_list)

In [ ]:
min(name_and_number.values())

97

In [9]:
def get_RGB(img_list, value):
  assert value in ["mean", "min", "max", "median"]
  R_s, G_s, B_s = [], [], []
  for img in img_list:
    function = eval(f"np.{value}")

    R, G, B = function(img[0]), function(img[1]), function(img[2])
    R_s.append(R), G_s.append(G), B_s.append(B)

  R, G, B = np.mean(R_s), np.mean(G_s), np.mean(B_s)
  return R, G, B

In [ ]:
name = name_set[0]
feature_list = {"R_mean": [], "G_mean": [], "B_mean": [], "R_median": [], "G_median": [], "B_median": [], "R_max": [], "G_max": [], "B_max": []}

for name in tqdm(name_set):
  img_list = glob(f"cropped/{name}*_cropped.jpg")[:97]
  imgs = [imageio.imread(img_file) for img_file in img_list]
  R, G, B = get_RGB(imgs, "mean")
  feature_list["R_mean"].append(R)
  feature_list["G_mean"].append(G)
  feature_list["B_mean"].append(B)

  R, G, B = get_RGB(imgs, "median")
  feature_list["R_median"].append(R)
  feature_list["G_median"].append(G)
  feature_list["B_median"].append(B)

  R, G, B = get_RGB(imgs, "max")
  feature_list["R_max"].append(R)
  feature_list["G_max"].append(G)
  feature_list["B_max"].append(B)

  0%|          | 0/90 [00:00<?, ?it/s]<ipython-input-10-88b57cda241d>:6: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  imgs = [imageio.imread(img_file) for img_file in img_list]
 68%|██████▊   | 61/90 [1:56:05<47:14, 97.74s/it]   

In [ ]:
model = R, G, B

In [ ]:
model = imageio.imread(img_list[1])[0])
np.median(imageio.imread(img_list[1])[1])
np.median(imageio.imread(img_list[2])[2])

<ipython-input-115-d179324259d3>:1: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  np.median(imageio.imread(img_list[1])[0])


0.0

<ipython-input-117-4a5cacfdb490>:1: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  np.median(imageio.imread(img_list[2])[2].max())


128.0